In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import os

model_name = "meta-llama/Llama-3.2-1B"

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# Load your dataset
dataset = load_dataset("json", data_files="./data/stories.json")

# Merge prompt + story for training
def preprocess(example):
    full_text = f"<s>Prompt: {example['prompt']}\nStory: {example['story']}</s>"
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=512)

tokenized = dataset["train"].map(preprocess)

# Training args
args = TrainingArguments(
    output_dir="./llama-story",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=20,
    save_steps=100,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
)

trainer.train()


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_dir = "./llama-story"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_story(prompt: str):
    input_text = f"<s>Prompt: {prompt}\nStory:"
    output = pipe(input_text, max_new_tokens=300, do_sample=True, temperature=0.8)
    return output[0]["generated_text"].replace(input_text, "").strip()

# Test
if __name__ == "__main__":
    prompt = "Kể câu chuyện về một con mèo đi lạc trong không gian"
    story = generate_story(prompt)
    print("Generated story:\n", story)


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from generate import generate_story

app = FastAPI()

class PromptInput(BaseModel):
    prompt: str

@app.post("/generate")
def generate(input: PromptInput):
    story = generate_story(input.prompt)
    return {"story": story}


In [ ]:
[
  {
    "prompt": "Kể câu chuyện về một cậu bé và con rồng",
    "story": "Ngày xửa ngày xưa, có một cậu bé tên An sống gần một hang động nơi con rồng đang ngủ..."
  },
  {
    "prompt": "Viết truyện cổ tích về nàng tiên cá",
    "story": "Trong lòng đại dương xanh thẳm, có một nàng tiên cá tên Lyra..."
  }
]
